## Detour notebook!
Here I mostly wanted to look at differneces between audience and critic preferences.

In [1]:
import pandas as pd

In [2]:
audience = pd.read_pickle('audience_table.pkl')['audience']

In [3]:
critic = pd.read_pickle('critic_table.pkl')

Add in the audience data.

In [4]:
critic['audience'] = (audience>60).astype(int)

Get the features we've been working with.

In [5]:
num_genres = critic.loc[:,'Action':'Western'].sum(axis=1)
num_genres = num_genres.rename('num_genres')
critic_table = critic.join(num_genres)

critic = pd.get_dummies(critic,columns = ['rating','decade','aspect_ratio','language','country'],drop_first=True)
critic = critic.drop(['rating','year','genre','title'],axis=1,errors='ignore')

critic['starpower'] = critic.post_face * (critic_table.actor_1_fb+critic_table.actor_2_fb+critic_table.actor_3_fb)

Set critic to 1 where it is Fresh.

In [6]:
critic['critic'] = (critic>60).astype(int)

Now let's look at all of our categoricals to see what makes a difference for critics but not audiences or vice versa!

In [8]:
cate = ['Action', 'Adventure',
       'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama',
       'Family', 'Fantasy', 'Film-Noir', 'Game-Show', 'History', 'Horror',
       'Music', 'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance',
       'Sci-Fi', 'Short', 'Sport', 'Thriller', 'War', 'Western',
       'rating_PG', 'rating_PG-13', 'rating_R', 'rating_andre', 'decade_10s',
       'decade_70s', 'decade_80s', 'decade_90s', 'decade_old', 'decade_ukjent',
       'aspect_ratio_2.35', 'aspect_ratio_other', 'language_andre',
       'country_France', 'country_UK', 'country_USA', 'country_andre']

Unexpectedly few things, and all genres.

In [17]:
genre_love = []
for cat in cate :
    category = critic[critic[cat]==1]
    num_films = len(category)
    
    c_liked = len(category[category.critic==1])
    c_percent_liked = c_liked/num_films
    
    a_liked = len(category[category.audience==1])
    a_percent_liked = a_liked/num_films
    
    if len(critic[critic[cat]==1]) > 100 :
        genre_love.append((cat,c_percent_liked,a_percent_liked))

    if abs(c_percent_liked-a_percent_liked)>.15 :
            print(cat)
            print(c_percent_liked)
            print(a_percent_liked)

Music
0.5404040404040404
0.7121212121212122
Musical
0.5877192982456141
0.7719298245614035
Sport
0.4785276073619632
0.7239263803680982


### Get a dataframe of the genre_love table so that we can save it and use it in visualisations.

In [31]:
df = pd.DataFrame(genre_love)

I want to focus on the genres, not the other categoricals.

In [24]:
df = df.iloc[:20,:]

In [28]:
df.to_csv('genre_loves.csv',header=False,index=False)